# Projet TAL M1 S1 : POS Tagging
structure du projet expliqué ici : https://u-paris.zoom.us/rec/share/rFB-gp7vkgMK31kfPxEsyNF2k6q7mHcq3P9MQiovimkr9ebIewZpuD62RBk6SmM.31Mmzx1VyMqQXRtu?startTime=1607605609000

# Implémentation du classifieur

## Import du corpus
Cours Zoom du 17/12 - 35 min

On charge les trois corpus *in-domain* sous la forme de listes de dictionnaires : chaque phrase a une clé "mots" qui est associée à une liste des mots, et une clé "POS" associée à une liste de POS. 

On utilise les trois corpus distincts de French-GSD :
- Train : apprentissage.
- Dev : validation. Pour tester et améliorer le modèle. 
- Test : évaluation. On ne l'utilisera pas pendant l'apprentissage ou le test.

In [9]:
# clés des dictionnaires
mot = "mot"
gold = "gold_label"
mot_prec = "préc"
mot_suiv = "suiv"
maj = "maj"
num = "nb"
nonAlphanum = "non alphanum"
long = "long"
court = "court"
un_car = "un caractère"
suff_adv = "suffixe adv"
suff_vb = "suffixe verb"
suff_noun = "suffixe nom"
suff_adj = "suffixe adj"


In [10]:
# j'ai juste rajouté l'encodage car j'avais une erreur sans 

def load_corpus(file):
    with open(file, "r", encoding = "utf8") as f: 
        content = f.read() # chargement du corpus
    content = content.split("\n\n") # séparation en phrases
    corpus = []
    for phrase in content: # pour chaque phrase
        phrase_dico = {"mots" : [], "gold_labels" : [], "lemme" : []} # liste qui contiendra 1 dictionnaire par mot de la phrase
        for line in phrase.splitlines():
            if not line.startswith("#"): # on ignore les lignes qui commencent par #
                features = line.split("\t")
                phrase_dico["mots"].append(features[1])
                phrase_dico["lemme"].append(features[2])
                phrase_dico["gold_labels"].append(features[3])
        corpus.append(phrase_dico)
    return corpus

gsd_train = load_corpus("corpus-in-domain/fr_gsd-ud-train.conllu")
gsd_test = load_corpus("corpus-in-domain/fr_gsd-ud-test.conllu")
gsd_dev = load_corpus("corpus-in-domain/fr_gsd-ud-dev.conllu")

In [11]:
print("---- Aperçus d'une phrase de chaque corpus-----", end="\n\n")
print(gsd_train[1], end="\n\n")
print(gsd_test[102], end="\n\n")
print(gsd_dev[564])

---- Aperçus d'une phrase de chaque corpus-----

{'mots': ["L'", 'œuvre', 'est', 'située', 'dans', 'la', 'galerie', 'des', 'de', 'les', 'batailles', ',', 'dans', 'le', 'château', 'de', 'Versailles', '.'], 'gold_labels': ['DET', 'NOUN', 'AUX', 'VERB', 'ADP', 'DET', 'NOUN', '_', 'ADP', 'DET', 'NOUN', 'PUNCT', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT'], 'lemme': ['le', 'œuvre', 'être', 'situer', 'dans', 'le', 'galerie', '_', 'de', 'le', 'bataille', ',', 'dans', 'le', 'château', 'de', 'Versailles', '.']}

{'mots': ['La', 'gestion', 'et', "l'", 'exploitation', 'de', 'la', 'salle', 'de', 'concert', 'Wagram', ',', 'récemment', 'rénovée', ',', 'sera', 'assurée', 'par', 'Eurosites', ',', 'leader', 'en', 'France', 'de', 'la', 'location', 'de', 'salles', '.'], 'gold_labels': ['DET', 'NOUN', 'CCONJ', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', 'PROPN', 'PUNCT', 'ADV', 'VERB', 'PUNCT', 'AUX', 'VERB', 'ADP', 'PROPN', 'PUNCT', 'NOUN', 'ADP', 'PROPN', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', '

## Extraction des caractéristiques

La fonction feature_extraction renvoie une liste de dictionnaires (un par mot) qui contiennent les caractéristiques suivantes pour chaque mot :
- mot
- mot précédent : pour le premier mot de la phrase son mot précédent sera "START", ce qui permettra de prendre en compte la caractéristique "être le premier mot".
- mot suivant : pour le dernier mot, ce sera "END".
- commence par une lettre majuscule
- est numérique
- contient des caractères non alphanumériques
- longueur du mot : redéfinir 3 caractéristiques booléens/binaires : a 1 seule caractère, a moins de 3 caractères, a plus de 3 caractères.
- a un suffixe nominal
- a un suffixe adjectival
- a un suffixe verbal (ça marche que sur les lemmes !)
- a un suffixe adverbial (là ça marche sur la forme fléchie)

Met-on dans les listes les suffixes qui appartiennent à 2 catégories ?
Peut-on utiliser les lemmes ?

On pourrait ajouter des suffixes aussi. Par exemple une feature pourrait être "finir en -ment", une autre suffi_nominaux (-al, -ion, etc.), etc. 
On est confrontées à un problème d'homonymie avec certains suffixes. Par exemple, -ains dans certains (pronom) et hautains (adj). A voir comment gérer ça. 

On ne conserve plus la structure des phrases, qui n'est plus nécessaire une fois qu'on a extrait les informations comme mot précédent et mot suivant.

In [12]:
def feature_extraction(corpus):

    corpus_features = []
    """
    Listes à utiliser pour les lemmes :
    list_adj = ["ain", "aine","aire","é", "ée","iel", "uel", "lle",  "al", "ales", "al", "ial","er", "ère", "ier", "esque", "eur", "euse", "ieux","ueux", "if","ive", "in","ine","ique","atoire", "u","ue", "issime", "able","ible", "uble", "ième","uple"]
    list_noun = ["ade", "age","aille", "aison", "ison","oison", "ation", "ition","ssion", "sion","xion", "isation","ment", "ement","erie", "ure","ature","at", "ance","ence", "escence","ité", "eté","té", "ie","erie", "esse","ise", "eur","isme", "iste", "seur", "isseur", "isateur", "euse","isseuse", "atrice","ier","ière", "aire","ien", ienne","iste",er","eron","eronne","trice","oir", "oire","ier","ière","erie","anderie", "aire","ain","aille", "ée", "ard","asse", "assier","âtre","aut","eau","ceau", "ereau","eteau", "elle", "et","elet","ette", 
"elette","in","otin", "ine" "illon","on", "ille", "erole","ole","iche"]
    """

    list_vb = ["iser","ifier", "oyer","ailler", "asser","eler", "eter","iller", "iner","nicher", "ocher","onner","otter","oter", "ouiller"]
    list_adj = ["ain", "aine","ains", "aines","aire", "aires","é", "ée","ées", "és","iel", "iels","uel", "uels", 
"lle", "lles","els", "el" "al", "ales", "al", "ial", "aux","iaux", "er","ers", "ère","ères", "ier", "iers",      "esque","esques", "eur","eurs", "euse","euses", "ieux","ueux", "if", "ifs","ive", "ives","in", "ins","ine",      "ines","iques", "ique","atoire", "u","ue", "us","ues", "issime","issimes","able","ible", "ibles","ables",  
    "uble","ubles", "ième","ièmes", "uple"]
    list_noun = ["ade", "ades", "age", "ages","aille", "ailles", "aison", "ison", "isons","oison", "ation", "itions", "ition", "ssion", "sion","xion", "isation","ment", "ement","erie", "eries","ure","ures","ature", "atures","at", "ance","ence", "escence","ité", "eté","té", "ie","erie", "esse", "ise", "eur","isme", "iste", "istes","eurs", "seur","seurs", "isseur","isseurs", "isateur","euse", "euses","isseuse", "isseuses",       "atrice", "atrices","ier", "iers","ière", "ières","aire","aires","ien", "iens","ienne", "iennes","iste",         "istes","er", "ers","eron", "erons","eronne","trice","oir", "oire","oires", "oirs","ier", "iers","ière",         "ières","erie","eries","anderie","aire", "aires","ain", "aines", "ée","ées","aille", "ard","asse", "asses", "assier","âtre","aut","eau", "eaux","ceau", "ereau","eteau", "elle","elles", "et","elet","ets","ette","elette","ettes", "elettes","in", "ins","otin", "ine","ines", "illon","on","ons","ille", "erole","eroles", "ole","oles", "iche"]

    for phrase in corpus: # ajout des features additionnelles
        for prev, word, suiv in zip(["START"] + phrase["mots"][:-1], phrase["mots"], phrase["mots"][1:] + ["END"]):
            # création de triplets (mot précédent, mot, mot suivant)
            # avec "START" en prev pour le 1er mot
            # et "END" en suiv pour le dernier

            lemma = phrase["lemme"][phrase["mots"].index(word)] 

            # ajout d'un dictionnaire de features du mot au corpus
            corpus_features.append({mot : word, 
                gold : phrase["gold_labels"][phrase["mots"].index(word)], # on récupère le gold_label correspondant
                mot_prec : prev,
                mot_suiv : suiv,
                num : word.isnumeric(),
                nonAlphanum : not word.isalnum(),
                long : len(word) < 3,
                court : len(word) >= 3,
                un_car : len(word) == 1,
                suff_adv : word.endswith("ment"),
                suff_noun : any(word.endswith(elem) and len(word) != len(elem) for elem in list_noun),
                suff_adj : any(word.endswith(elem) and len(word) != len(elem) for elem in list_adj),
                suff_vb : any(word.endswith(elem) for elem in list_vb)
                # on vérifie la longueur du mot pour être sûr que ce soit un suffixe car on peut avoir le mot                      age avec le suffixe age par exemple ou bien aux
                # suff_noun : any(lemma.endswith(elem) and len(word) != len(elem) for elem in list_noun),
                # suff_adj : any(lemma.endswith(elem) for elem in list_adj),
                # suff_vb : any(lemma.endswith(elem) for elem in list_vb)
                
                })

    return corpus_features # renvoie les features transformés en vecteurs one-hot


gsd_train_features = feature_extraction(gsd_train)
gsd_dev_features = feature_extraction(gsd_dev)
gsd_test_features = feature_extraction(gsd_test)

In [13]:
print(*gsd_train_features[:10], sep="\n")
print(len(gsd_train_features))

{'mot': 'Les', 'gold_label': 'DET', 'préc': 'START', 'suiv': 'commotions', 'nb': False, 'non alphanum': False, 'long': False, 'court': True, 'un caractère': False, 'suffixe adv': False, 'suffixe nom': False, 'suffixe adj': False, 'suffixe verb': False}
{'mot': 'commotions', 'gold_label': 'NOUN', 'préc': 'Les', 'suiv': 'cérébrales', 'nb': False, 'non alphanum': False, 'long': False, 'court': True, 'un caractère': False, 'suffixe adv': False, 'suffixe nom': True, 'suffixe adj': False, 'suffixe verb': False}
{'mot': 'cérébrales', 'gold_label': 'ADJ', 'préc': 'commotions', 'suiv': 'sont', 'nb': False, 'non alphanum': False, 'long': False, 'court': True, 'un caractère': False, 'suffixe adv': False, 'suffixe nom': False, 'suffixe adj': True, 'suffixe verb': False}
{'mot': 'sont', 'gold_label': 'AUX', 'préc': 'cérébrales', 'suiv': 'devenu', 'nb': False, 'non alphanum': False, 'long': False, 'court': True, 'un caractère': False, 'suffixe adv': False, 'suffixe nom': False, 'suffixe adj': False,

## Implémentation de l'algorithme de classification
On a choisi d'implémenter la classification avec un perceptron moyenné.

Etiquettes du corpus : ADJ, ADP, ADV, AUX, CCONJ, DET, INTJ, NOUN, NUM, PART, PRON, PROPN, PUNCT, SCONJ, SYM, VERB, X

Sources : cours 3/12 + perceptron.pdf + zoom du 17/12

Je comprends pas comment on doit gérer les mots : comment on peut les ajouter ou les enlever de nos vecteurs ?!

In [14]:
def perceptron_train(training_set, MAX_EPOCH=10):
    
    tags = ["ADJ", "ADP", "ADV", "AUX", "CCONJ", "DET", "INTJ", "NOUN", "NUM", "PART", "PRON", "PROPN", "PUNCT", "SCONJ", "SYM", "VERB", "X"]

    # TODO initialisation des poids : à zéros ou bien de manière aléatoire ??
    a = [[0]*len(training_set[0]) for tag in tags] # poids moyennés
    # un vecteur de poids par classe - plutôt un dictionnaire ? FIXME

    for i in range(0, MAX_EPOCH):
        w = [[0]*len(training_set[0]) for tag in tags] # initialisé du vecteur de poids
        shuffled_set = shuffle(training_set.copy()) # on mélange le training set

        for phrase in shuffled_set:
            for word in phrase:
                prediction = max()
                # on cherche le tag le plus probable... mais comment ?... 
                
                # on compare avec le gold_label
                # si mauvaise prédiction
                    #  on ajoute x(i) à chaque poids de l'étiquette correcte
                    #  on retire x(i) à chaque poids de l'étiquette mal prédite

        # on ajoute w à a
    return a

poids_gsd_train = perceptron_train(gsd_train_features)

NameError: name 'shuffle' is not defined

## Evaluation des performances sur le corpus de validation (Dev)
Il faudra bien tester les hyper paramètres (epoch...)

## Evaluation sur le corpus Test

# Evaluation hors-domaine

## Analyse de l'impact du changement de domaine
* identifier causes de la baisse de performance : analyse de sortie, matrice de confusion, erreurs les + fréquentes
* réfléchir à des caractéristiques plus adaptées
* sélection d'un nouvel ensemble d'apprentissage avec des exemples représentatif (généré par un modèle de langue type TP2)

### Corpus UGC (User-generated content)

### Corpus littéraire

## Développement de systèmes robutes au changement de domaine